# Explore here

Paso 1: Busca un conjunto de datos
Investiga en las diferentes fuentes en línea sobre distintos datasets que podrías utilizar para entrenar un modelo. Puedes utilizar Kaggle de conjuntos de datos, entre otras muchas fuentes. 

In [22]:
# Your code here
import pandas as pd
import warnings

df = pd.read_csv('G:\Mi unidad\Data Science\Practicas\Tutorial-de-aplicacion-web-de-ml-usando-flask/dataset.csv')
df.head()

<>:5: SyntaxWarning: invalid escape sequence '\M'
<>:5: SyntaxWarning: invalid escape sequence '\M'
C:\Users\rgarciamontero\AppData\Local\Temp\ipykernel_18964\3706565685.py:5: SyntaxWarning: invalid escape sequence '\M'
  df = pd.read_csv('G:\Mi unidad\Data Science\Practicas\Tutorial-de-aplicacion-web-de-ml-usando-flask/dataset.csv')


,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Nacionality,Mother's qualification,Father's qualification,Mother's occupation,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,1,8,5,2,1,1,1,13,10,6,...,0,0,0,0,0.000000,0,10.8,1.4,1.74,Dropout
1,1,6,1,11,1,1,1,1,3,4,...,0,6,6,6,13.666667,0,13.9,-0.3,0.79,Graduate
2,1,1,5,5,1,1,1,22,27,10,...,0,6,0,0,0.000000,0,10.8,1.4,1.74,Dropout
3,1,8,2,15,1,1,1,23,27,6,...,0,6,10,5,12.400000,0,9.4,-0.8,-3.12,Graduate
4,2,12,1,3,0,1,1,22,28,10,...,0,6,6,6,13.000000,0,13.9,-0.3,0.79,Graduate


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4424 entries, 0 to 4423
Data columns (total 35 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   Marital status                                  4424 non-null   int64  
 1   Application mode                                4424 non-null   int64  
 2   Application order                               4424 non-null   int64  
 3   Course                                          4424 non-null   int64  
 4   Daytime/evening attendance                      4424 non-null   int64  
 5   Previous qualification                          4424 non-null   int64  
 6   Nacionality                                     4424 non-null   int64  
 7   Mother's qualification                          4424 non-null   int64  
 8   Father's qualification                          4424 non-null   int64  
 9   Mother's occupation                      

In [24]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Separar características y objetivo
X = df.drop('Target', axis=1)
y = df['Target']

# Dividir datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocesamiento
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

numerical_pipeline = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_pipeline = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_pipeline, numerical_features),
        ('cat', categorical_pipeline, categorical_features)
    ])

# Aplicar preprocesamiento
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)


Paso 2: Desarrolla un modelo
Una vez hayas encontrado tu conjunto de datos ideal, analízalo y entrena un modelo. Optimízalo si fuera necesario.

In [25]:
# Entrenamiento del Modelo
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Entrenar un modelo básico
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Evaluación
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

     Dropout       0.84      0.76      0.80       316
    Enrolled       0.54      0.34      0.41       151
    Graduate       0.78      0.94      0.85       418

    accuracy                           0.77       885
   macro avg       0.72      0.68      0.69       885
weighted avg       0.76      0.77      0.76       885



In [26]:
# Optimizacion del modelo

from sklearn.model_selection import GridSearchCV

# Definir parámetros a ajustar
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30]
}

# Búsqueda de cuadrícula con validación cruzada
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train)

print("Mejor puntaje:", grid_search.best_score_)
print("Mejores parámetros:", grid_search.best_params_)


Mejor puntaje: 0.7756494777807078
Mejores parámetros: {'max_depth': None, 'n_estimators': 100}


In [27]:
# Evaluar el modelo optimizado
best_model = grid_search.best_estimator_
y_test_pred = best_model.predict(X_test)
print(classification_report(y_test, y_test_pred))


              precision    recall  f1-score   support

     Dropout       0.84      0.76      0.80       316
    Enrolled       0.54      0.34      0.41       151
    Graduate       0.78      0.94      0.85       418

    accuracy                           0.77       885
   macro avg       0.72      0.68      0.69       885
weighted avg       0.76      0.77      0.76       885



In [28]:
# características más importantes para las decisiones del modelo, lo cual es útil para la interpretación.
importances = best_model.feature_importances_
feature_names = X.columns
feature_importance_df = pd.DataFrame({'feature': feature_names, 'importance': importances})
print(feature_importance_df.sort_values(by='importance', ascending=False))


                                           feature  importance
28             Curricular units 2nd sem (approved)    0.139524
29                Curricular units 2nd sem (grade)    0.108936
22             Curricular units 1st sem (approved)    0.093029
23                Curricular units 1st sem (grade)    0.074309
27          Curricular units 2nd sem (evaluations)    0.048938
17                               Age at enrollment    0.042033
21          Curricular units 1st sem (evaluations)    0.039941
14                         Tuition fees up to date    0.038683
3                                           Course    0.037771
10                             Father's occupation    0.033178
9                              Mother's occupation    0.030291
33                                             GDP    0.027255
31                               Unemployment rate    0.025785
8                           Father's qualification    0.025599
7                           Mother's qualification    0

Paso 3: Desarrolla una aplicación web usando Flask
Con los conocimientos adquiridos en este módulo, desarrolla una interfaz para poder utilizar el modelo. Dale el estilo que más te convenga y anota los recursos externos que hayas utilizado para el desarrollo.

In [29]:
pip install Flask

Note: you may need to restart the kernel to use updated packages.


In [30]:
from flask import Flask, request, render_template
import joblib

app = Flask(__name__)

# Manejo de excepciones al cargar el modelo
try:
    model_path = 'G:\Mi unidad\Data Science\Practicas\Tutorial-de-aplicacion-web-de-ml-usando-flask\models/modelo_entrenado.pkl'
    print(f"Intentando cargar el modelo desde: {model_path}")
    model = joblib.load(model_path)
    print("Modelo cargado correctamente.")
except FileNotFoundError as e:
    print(f"Archivo no encontrado: {e}")
    exit(1)  # Salir con código de error
except Exception as e:
    print(f"Ocurrió un error al cargar el modelo: {e}")
    exit(1)  # Salir con código de error

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    data = request.form.to_dict()
    input_data = [list(data.values())]  # Procesar como necesites
    prediction = model.predict(input_data)
    return render_template('index.html', prediction_text='Predicción: {}'.format(prediction))

if __name__ == "__main__":
    app.run(debug=True)



Intentando cargar el modelo desde: G:\Mi unidad\Data Science\Practicas\Tutorial-de-aplicacion-web-de-ml-usando-flask\models/modelo_entrenado.pkl
Modelo cargado correctamente.
 * Serving Flask app '__main__'
 * Debug mode: on


<>:8: SyntaxWarning: invalid escape sequence '\M'
<>:8: SyntaxWarning: invalid escape sequence '\M'
C:\Users\rgarciamontero\AppData\Local\Temp\ipykernel_18964\3441614317.py:8: SyntaxWarning: invalid escape sequence '\M'
  model_path = 'G:\Mi unidad\Data Science\Practicas\Tutorial-de-aplicacion-web-de-ml-usando-flask\models/modelo_entrenado.pkl'
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

g:\Mi unidad\Data Science\Practicas\Tutorial-de-aplicacion-web-de-ml-usando-flask\primer_modelo_flask\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
